In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [3]:
# Load dataset 
df = pd.read_csv("Churn_Modelling.csv")

In [4]:
print(df.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

In [5]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
print("Number of row:",df.shape[0])
print("Number of columns:",df.shape[1])

Number of row: 10000
Number of columns: 14


In [7]:
# Droping the irrelevent c(olumns)features 
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [8]:
# Encoding the Categorical varibles 
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [9]:
from sklearn.preprocessing import OneHotEncoder
one_hot_geo=OneHotEncoder()
geo_encoder= one_hot_geo.fit_transform(df[['Geography']])

In [10]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
new=one_hot_geo.get_feature_names_out(['Geography'])

In [12]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(),columns=new)

In [13]:
# combined one hot encoded columns with original data

df=pd.concat([df.drop('Geography',axis=1),geo_encoder_df],axis=1)

df.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
X = df.drop('EstimatedSalary',axis=1)
y = df['EstimatedSalary']

In [15]:
#Spliting the data set into training and testing 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [16]:
# Scale these features 
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [17]:
# save the Encoder and scaler 
with open('label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder_gender,file)


with open('one_hot_geo.pkl','wb')as file:
    pickle.dump(one_hot_geo,file)


with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

### ANN with Regression Problem 

In [18]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [19]:
#Build the ANN model 
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), # HL1 connected with i/p layer
    Dense(32,activation='relu'), # HL 2 layer
    Dense(1) # out put layer Defalut activation function is Linear regression function 
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Define the optimizer and loss function
import tensorflow 
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
losses = tensorflow.keras.losses.MeanAbsoluteError()

In [23]:
model.compile(optimizer=opt, loss=losses, metrics=['mae'])

In [24]:
# Set up the Tensorboard 

log_dir='regressionlog/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [27]:
# Early stopping callback to prevent overfitting (based on validation loss)
from tensorflow.keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',   # Monitor validation loss
    patience=20,           # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore the model weights from the best epoch
)


In [28]:
# Train model 
histroy = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 49454.7227 - mae: 49454.7227 - val_loss: 50303.0039 - val_mae: 50303.0039
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 49424.5703 - mae: 49424.5703 - val_loss: 50437.5430 - val_mae: 50437.5430
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 49404.3477 - mae: 49404.3477 - val_loss: 50291.1641 - val_mae: 50291.1641
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 49352.2773 - mae: 49352.2773 - val_loss: 50361.7773 - val_mae: 50361.7773
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 49385.1953 - mae: 49385.1953 - val_loss: 50491.4688 - val_mae: 50491.4688
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 49344.0391 - mae: 49344.0391 - val_loss: 50556.6914 - val_mae: 50556.6914
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 49316.2656 - mae: 49316.2656 - 

In [33]:
# Model Evalution on test data 
test_loss, test_mae =model.evaluate(X_test,y_test)
print(f'The MAE :' ,test_loss)

63/63 [==============================] - 0s 2ms/step - loss: 50291.1641 - mae: 50291.1641
The MAE : 50291.1640625


In [34]:
model.save('regression_model.h6')

INFO:tensorflow:Assets written to: regression_model.h6\assets


INFO:tensorflow:Assets written to: regression_model.h6\assets
